In [1]:
import math
import random 

import gym
import numpy as np
        
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.distributions import Normal,Beta
from sklearn import preprocessing

from IPython.display import clear_output
import matplotlib.pyplot as plt
%matplotlib inline

seed_number = 10

<h2>Use CUDA</h2>

In [2]:
np.random.seed(seed_number)
torch.backends.cudnn.deterministic = True
torch.manual_seed(seed_number)

use_cuda = torch.cuda.is_available()

if use_cuda:
    torch.cuda.manual_seed_all(seed_number)
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

<h2>Neural Network</h2>

In [3]:
def init_weights(m):
    if isinstance(m, nn.Linear):
        nn.init.normal_(m.weight, mean=0., std=0.1)
        nn.init.constant_(m.bias, 0.1)
        

class ActorCritic(nn.Module):
    def __init__(self, num_inputs, num_outputs, hidden_size, std=0.0):
        super(ActorCritic, self).__init__()
        
        self.critic = nn.Sequential(
            nn.Linear(num_inputs, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, 1)
        )
        
        self.actor = nn.Sequential(
            nn.Linear(num_inputs, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, num_outputs)
        )
        self.log_std = nn.Parameter(torch.ones(1, num_outputs) * std)
        
        self.apply(init_weights)
        
    def forward(self, x):
        value = self.critic(x)
        mu    = self.actor(x)
        std   = self.log_std.exp().expand_as(mu)
        dist  = Normal(mu, std)
        return dist, value

class ModelBasedGoalNetwork(nn.Module):
    def __init__(self, num_inputs, num_outputs, hidden_size):
        super(ModelBasedGoalNetwork, self).__init__()
        
        self.goalnetwork = nn.Sequential(
            nn.Linear(num_inputs, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, num_outputs)
        )
    
    def forward(self, x):
        subgoal = self.goalnetwork(x)
        return subgoal

In [4]:
def plot(frame_idx, rewards):
    clear_output(True)
    plt.figure(figsize=(20,5))
    plt.subplot(131)
    plt.title('frame %s. reward: %s' % (frame_idx, rewards[-1]))
    plt.plot(rewards)
    plt.show()
    
def test_env(model, goal, vis=False):
    state = env.reset()
    if vis: env.render()
    done = False
    total_reward = 0
    while not done:
        state_goal = np.concatenate((state,goal),0)
        state_goal = torch.FloatTensor(state_goal).unsqueeze(0).to(device)
        dist, _ = model(state_goal)
        next_state, reward, done, _ = env.step(dist.sample().cpu().numpy()[0])
        state = next_state
        if vis: env.render()
        total_reward += reward
    return total_reward

<h2>Hindsight GAE</h2>

In [5]:
def compute_gae(next_value, rewards, masks, values, gamma=0.99, lamda=0.95):
    values = values + [next_value]
    gae = 0
    returns = []
    
    for step in reversed(range(len(rewards))):
        delta = rewards[step] + gamma * values[step + 1] * masks[step] - values[step]
        gae = delta + gamma * lamda * masks[step] * gae
        returns.insert(0, gae + values[step])
    return returns

<h2> Importance Hindsight GAE </h2>

In [6]:
def hindsight_gae(rewards, current_logprobs, desired_logprobs, masks, values, gamma = 0.995, lamda = 0):
    lambda_ret = 1
    hindsight_gae = 0
    returns = []
    
    for step in range(len(rewards)):
        temp = 0
        is_weight_ratio = 1
        for step_ in range(step, len(rewards)):
            ratio = (current_logprobs[step_] - desired_logprobs[step_]).exp() 
            clipped_ratio = lambda_ret * torch.clamp(ratio, max = 1)
            is_weight_ratio = is_weight_ratio * clipped_ratio
        for step_ in range(step, len(rewards)):
            temp = temp + ((gamma ** (step_+1)) * rewards[step_] - (gamma ** (step_)) * rewards[step_])  
        temp = temp - (gamma ** (step + 1)) * rewards[step]
        
        delta = rewards[step] + is_weight_ratio * temp
        hindsight_gae = delta + gamma * lamda * masks[step] * hindsight_gae
        returns.insert(0, hindsight_gae + values[step])
        
    return returns

<h2> Compute Return </h2>

In [7]:
def compute_returns(rewards, gamma = 0.995):
    returns = 0
    returns_list = []
    for step in range(len(rewards)):
        returns = returns + (gamma ** i) * rewards[step] 
        returns_list.insert(0,returns)
    return returns_list

<h1> Proximal Policy Optimization Algorithm</h1>
<h2><a href="https://arxiv.org/abs/1707.06347">Arxiv</a></h2>

In [8]:
def ppo_iter(mini_batch_size, states, actions, log_probs, returns, advantage):
    batch_size = states.size(0)

    for _ in range(batch_size // mini_batch_size):
        rand_ids = np.random.randint(0, batch_size, mini_batch_size)
        yield states[rand_ids, :],actions[rand_ids,:],log_probs[rand_ids,:],returns[rand_ids,:],advantage[rand_ids,:]
        
        

def ppo_update(ppo_epochs, mini_batch_size, states, actions, log_probs, returns, 
               advantages, episode_count, test_reward, best_return, clip_param=0.2):
    actor_loss_list = []
    critic_loss_list = []
    clip = 5
    for ppo_epoch in range(ppo_epochs):
        for state, action, old_log_probs, return_, advantage in ppo_iter(mini_batch_size, states, 
                                                                         actions, log_probs, returns, advantages):
            model.zero_grad()
            dist, value = model(state)

            entropy = dist.entropy().mean()
            new_log_probs = dist.log_prob(action)

            ratio = (new_log_probs - old_log_probs).exp()

            surr1 = ratio * advantage
            surr2 = torch.clamp(ratio, 1.0 - clip_param, 1.0 + clip_param) * advantage

            actor_loss  = - torch.min(surr1, surr2).mean()
            # MSE Loss
            critic_loss = (return_ - value).pow(2).mean() 
            
            # Huber Loss
            # critic_loss = nn.functional.smooth_l1_loss(value, return_)
            
            actor_loss_list.append(actor_loss.data.cpu().numpy().item(0))
            critic_loss_list.append(critic_loss.data.cpu().numpy().item(0))
            
            loss = 0.5 * critic_loss + actor_loss - 0.0001 * entropy
            
            optimizer.zero_grad()
            loss.backward()
            
            # clip gradient to prevent gradient exploding
            nn.utils.clip_grad_norm_(model.parameters(), clip)
            
            optimizer.step()
    
    mean_actor_loss = np.mean(actor_loss_list)
    mean_critic_loss = np.mean(critic_loss_list)
    
    mean_actor_loss_list.append(mean_actor_loss)
    mean_critic_loss_list.append(mean_critic_loss)
    
    assert ~np.isnan(mean_critic_loss), "Assert error: critic loss has nan value." 
    assert ~np.isinf(mean_critic_loss), "Assert error: critic loss has inf value."

    print ('\nEpisode: {0}, actor_loss: {1:.3f}, critic_loss: {2:.3f}, mean_reward: {3:.3f}, best_return: {4:.3f}'
           .format(episode_count, mean_actor_loss, mean_critic_loss, test_reward, best_return))

# Create Environment

In [9]:
from multiprocessing_env import SubprocVecEnv

num_envs = 16
env_name = "Pendulum-v0"

def make_env(i):
    def _thunk():
        
        env = gym.make(env_name)
        env.seed(i+seed_number)
        return env

    return _thunk

envs = [make_env(i) for i in range(num_envs)]
envs = SubprocVecEnv(envs)

env = gym.make(env_name)
env.seed(seed_number)

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
WARN: gym.

[10]

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.


# Initial Goal Distribution Generation

In [10]:
class RandomAgent(object):
    def __init__(self, action_space):
        self.action_space = action_space

    def act(self, observation, reward, done):
        return self.action_space.sample()
    
agent = RandomAgent(env.action_space)

episode_count = 50
reward = 0
done = False
initial_subgoals = []
        
for i in range(episode_count):
    state = env.reset()
#     print (state)
    done_count = 0
    while True:
        action = agent.act(state, reward, done)
        next_state, reward, done, _ = env.step(action)
        state = next_state
        if done:
            break
    initial_subgoals.append(state)

In [11]:
random.seed(seed_number)
initial_subgoal = initial_subgoals[random.randint(0, len(initial_subgoals)-1)]
print ('Initial subgoal sampled is: ', initial_subgoal)

Initial subgoal sampled is:  [ 0.11320659 -0.99357147 -0.14532486]


<h2> Model Based Goal Generator </h2>

In [12]:
def generate_random_trajectories(num_traj):
    print ('generating random trajectories...')
    dataset_random = []

    game_rewards = []
    for n in range(num_traj):

        obs = env.reset()
        while True:
            sampled_action = env.action_space.sample()
            new_obs, reward, done, _ = env.step(sampled_action)

            dataset_random.append([obs, new_obs, reward, done, sampled_action])

            obs = new_obs
            game_rewards.append(reward)

            if done:
                break

    # print some stats
    print('mean rand_dataset reward:',np.round(np.sum(game_rewards)/num_traj,2), 
          'max rand_dataset reward:', np.round(np.max(game_rewards),2), np.round(len(game_rewards)/num_traj))

    return dataset_random

def flatten_rl_dataset(temp, rl_dataset):
    for i in range(num_steps):
        for j in range(num_envs):
            rl_dataset.append([temp[0][0][i][j],temp[0][1][i][j],temp[0][2][i][j].cpu().numpy(),
                               temp[0][3][i][j].astype('bool'),temp[0][4][i][j].cpu().numpy()])
    return rl_dataset

def filtered_prep(dataset, min_dist):
    new_dataset = []
    for i in range(len(dataset)):
        curr_dist = np.linalg.norm(dataset[i][0]-desired_goal)
        if (curr_dist > (min_dist - 0.2)) and (curr_dist < (min_dist + 0.2)):
            new_dataset.append(dataset[i])
    return np.array(new_dataset)
                    
def MSELoss(y_truth, y_pred):
    y_truth = torch.FloatTensor(np.array(y_truth)).to(device)
    return F.mse_loss(y_pred.view(-1).float(), y_truth.view(-1))


def goal_network_update(dataset, train_iter, goal_model, goal_optimizer):
    # split dataset to 80% training and 20% validation
    len_data = len(dataset)
    
    d_train  = dataset[:int(len_data * 0.8)]
    d_valid  = dataset[int(len_data * 0.8):-1]
    
    state_action_input = np.concatenate((dataset[-1][0],dataset[-1][4]),0)
    sff      = np.arange(len(d_train))
    np.random.shuffle(sff)
    d_train  = d_train[sff]
    
    # training dataset
    x_train  = np.array([np.concatenate([s,a]) for s,_,_,_,a in d_train]) 
    y_train  = np.array([ns for _,ns,_,_,_ in d_train])
    
    # validation dataset
    x_valid  = np.array([np.concatenate([s,a]) for s,_,_,_,a in d_valid])
    y_valid  = np.array([ns for _,ns,_,_,_ in d_valid])

    losses_goal = []
    mse_valid = []
    # go through max_model_iter supervised iterations
    for i in range(train_iter):
        x_train += np.random.normal(loc = 0, scale=0.001, size = x_train.shape)

        goal_optimizer.zero_grad()
        pred_goal = goal_model((torch.tensor(x_train)).type(torch.FloatTensor).to(device))
        goal_loss = MSELoss(y_train, pred_goal)
        # print ('goal_loss: ', goal_loss.cpu().detach().numpy())
        losses_goal.append(goal_loss.cpu().detach().numpy())
        goal_loss.backward()
        goal_optimizer.step()

        # iteratively evaluate
        if i % 5 == 0:
            goal_model.eval()
            pred_goal = goal_model((torch.tensor(x_valid)).type(torch.FloatTensor).to(device))
            goal_model.train(True)
            valid_goal_loss = MSELoss(y_valid, pred_goal)
            mse_valid.append(valid_goal_loss)
            # print ('evaluation iteration: ',i, ' ,valid_goal_loss: ', valid_goal_loss.cpu().detach().numpy())
    # final evaluation
    goal_model.eval()
    pred_goal = goal_model((torch.tensor(x_valid)).type(torch.FloatTensor).to(device))
    goal_model.train(True)
    valid_goal_loss = MSELoss(y_valid, pred_goal)
    mse_valid.append(valid_goal_loss)
    # print ('end of training validation goal loss: ', valid_goal_loss.cpu().detach().numpy())
    return losses_goal, mse_valid, state_action_input

def model_based_goal_generator(rand_dataset, rl_dataset, rand_num_traj, mb_train_iter, min_dist, 
                               goal_model, goal_optimizer):
    print ('generate goal using supervised model based...')

    
    if len(rl_dataset) > 0:
        concat_dataset    = np.concatenate([rand_dataset, rl_dataset], axis=0)
    else:
        concat_dataset    = np.array(rl_dataset)
    print ('len after concat dataset: ', len(concat_dataset))

    filtered_dataset  = filtered_prep (concat_dataset, min_dist)
    print ('len filtered concat dataset: ', len(filtered_dataset))
    # if filtered dataset empty
    if len(filtered_dataset) > 0:
        mb_loss, mse_eval, state_action_input = goal_network_update(filtered_dataset, mb_train_iter, 
                                                                goal_model, goal_optimizer)
        all_mb_loss.append(mb_loss)
        all_goal_mse_eval.append(mse_eval)

        model_based_goal = goal_model(torch.FloatTensor(state_action_input).to(device))

        return model_based_goal
    else:
        return [0]
def evaluate_goals(state, num_envs, goal, desired_goal):
    sf_subgoal = state[random.randint(0, num_envs - 1)]
    mb_distance = np.linalg.norm(goal - desired_goal)
    sf_distance = np.linalg.norm(sf_subgoal - desired_goal) 
    print ('mb subgoal: ', goal, ' distance: ', mb_distance)
    print ('sample final subgoal: ', sf_subgoal, 'distance: ', sf_distance)
    mb_distance_list.append(mb_distance)
    sf_distance_list.append(sf_distance)
    

# Training Agent

In [ ]:
num_inputs  = envs.observation_space.shape[0]
num_outputs = envs.action_space.shape[0]

# training hindsight ppo model hyperparameter
hidden_size      = 256
lr               = 3e-4
num_steps        = 20 # 20
mini_batch_size  = 5
ppo_epochs       = 4
threshold_reward = -200
max_frames       = 24000# 50000
frame_idx        = 0
episode_count    = 0
best_return      = -9999
desired_goal     = np.asarray([0,0,0])

# options hyperparameter
use_modelbased   = True
early_stop       = False
multi_goal       = True

# statistical variable
test_rewards          = []
mean_actor_loss_list  = []
mean_critic_loss_list = []

# model based parameter
rand_num_traj     = 1
mb_train_iter     = 60
init_goal_dist    = 7.0
end_goal_dist     = 0.01
decay_rate        = 1000 / math.log(700)
goal_network_lr   = 3e-4
all_mb_loss       = []
all_goal_mse_eval = []
mb_distance_list  = []
sf_distance_list  = []

# network model
model            = ActorCritic(2*num_inputs, num_outputs, hidden_size).to(device)
optimizer        = optim.Adam(model.parameters(), lr = lr)
goal_model       = ModelBasedGoalNetwork(num_inputs+num_outputs, num_inputs, hidden_size).to(device)
goal_optimizer   = optim.Adam(goal_model.parameters(), lr = goal_network_lr)

random_dataset   = generate_random_trajectories(rand_num_traj)
temp_rl_dataset  = []
rl_dataset       = []
state = envs.reset()

while frame_idx < max_frames:
    
    # sample state from previous episode
    if multi_goal:
        if frame_idx == 0: 
            goal = initial_subgoal
        else: 
            if use_modelbased:
                min_dist = init_goal_dist * math.exp((-1)*episode_count/decay_rate)
                goal = model_based_goal_generator(random_dataset, rl_dataset, rand_num_traj, mb_train_iter, 
                                                    min_dist,goal_model, goal_optimizer)
                if len(goal) > 1:
                    goal = goal.cpu().detach().numpy()
                    temp = goal
                else: 
                    goal = temp
                    
                if episode_count != 0:
                    evaluate_goals(state, num_envs, goal, desired_goal)
            else: 
                if len(state) > 1:
                    goal = state[random.randint(0, num_envs - 1)]
                else:
                    goal = state[0]
    else:
        goal = desired_goal
        
    log_probs         = []
    log_probs_desired = []
    values            = []
    states            = []
    states_goals      = []
    next_states       = []
    actions           = []
    actions_desired   = []
    rewards           = []
    dones             = []
    masks             = []
    entropy = 0

    for i in range(num_steps):
        state_goals = []
        state_desired_goals = []
        next_state_goals = []
        
        # append state with subgoal and desired goal
        for s in state: 
            state_goal = np.concatenate((s,goal),0)
            state_goals.append((state_goal))
            state_desired_goal = np.concatenate((s, desired_goal), 0)
            state_desired_goals.append((state_desired_goal))
            
        state_goals = np.array(state_goals)
        state_goals = torch.FloatTensor(state_goals).to(device)
        state_desired_goals = np.array(state_desired_goals)
        state_desired_goals = torch.FloatTensor(state_desired_goals).to(device)
        
        # for subgoal
        dist, value = model(state_goals)
        action = dist.sample() 
        next_state, reward, done, _ = envs.step(action.cpu().numpy())
        
        # for desired goal
        dist_desired, value_desired = model(state_desired_goals)
        action_desired = dist_desired.sample()
        
        # append next state with sub goal
        for n_s in next_state: 
            next_state_goal = np.concatenate((n_s, goal), 0)
            next_state_desired_goal = np.concatenate((n_s, desired_goal), 0)
            next_state_goals.append((next_state_goal)) 
        next_state_goals = np.array(next_state_goals)
        
        # for subgoal
        log_prob = dist.log_prob(action)
        # for desired goal
        log_prob_desired = dist_desired.log_prob(action_desired)
        
        entropy += dist.entropy().mean()
        
        # normalized reward
        reward = (reward - np.mean(reward))/(np.std(reward) + 1e-5)
        
        states.append(state)
        next_states.append(next_state)
        states_goals.append(state_goals)
        actions.append(action)
        actions_desired.append(action_desired)
        log_probs.append(log_prob)
        log_probs_desired.append(log_prob_desired)
        dones.append(done)
        rewards.append(torch.FloatTensor(reward).unsqueeze(1).to(device))
        masks.append(torch.FloatTensor(1 - done).unsqueeze(1).to(device))
        values.append(value)
        
        state = next_state
        frame_idx += 1
        
        if frame_idx % num_steps == 0:
            test_reward = np.mean([test_env(model, desired_goal) for _ in range(5)])
            test_rewards.append(test_reward)
            if test_reward >= best_return:
                best_return = test_reward
            # plot(frame_idx, test_rewards)
            if test_reward > threshold_reward: early_stop = True
                
    temp_rl_dataset.append([states, next_states, rewards, dones, actions])
    
    # forget older data (reduce bias)
    if episode_count % 20 == 0:
        rl_dataset = []
    rl_dataset = flatten_rl_dataset(temp_rl_dataset, rl_dataset)
    
    next_state_goals = torch.FloatTensor(next_state_goals).to(device)
    _, next_value = model(next_state_goals)

    old_logprobs     = log_probs 
    current_logprobs = log_probs_desired
    
    # print ('old_logprobs: ', log_probs)
    # print ('current_logprobs: ', current_logprobs)
    returns        = hindsight_gae(rewards, old_logprobs, current_logprobs, masks, values)
#     returns        = compute_gae (next_value, rewards, masks, values)
                          
    returns        = torch.cat(returns).detach()
    log_probs      = torch.cat(log_probs).detach()
    values         = torch.cat(values).detach()
    states_goals   = torch.cat(states_goals)
    actions        = torch.cat(actions)
    advantage      = returns - values

    ppo_update(ppo_epochs, mini_batch_size, states_goals, actions, log_probs, returns, advantage, episode_count, 
               test_reward, best_return)
    
    if frame_idx % (num_steps * 50) == 0:
        lower_bound = int((frame_idx - (num_steps * 50)) / num_steps)
        upper_bound = int(frame_idx / num_steps)
        last_fifty_episode_mean_reward = np.mean(test_rewards[lower_bound:upper_bound])
        print ('last 50 episode mean reward: ', last_fifty_episode_mean_reward)
        print ('\n')
    
    episode_count += 1

# Saving and Loading Testing Reward

In [21]:
import pickle

with open('./Test Reward Plot/mb_test_rewards2', 'wb') as fp1:
    pickle.dump(test_rewards, fp1)
with open('./Loss Plot/mb_mean_actor_loss2', 'wb') as fp2:
    pickle.dump(mean_actor_loss_list, fp2)
with open('./Loss Plot/mb_mean_critic_loss2', 'wb') as fp3:
    pickle.dump(mean_critic_loss_list, fp3)

In [22]:
with open('./Goal Plot/mb_goal4', 'wb') as fp1:
    pickle.dump(mb_distance_list, fp1)

with open('./Goal Plot/sf_goal4', 'wb') as fp1:
    pickle.dump(sf_distance_list, fp1)

<h1> Save and Load Model </h1>

In [16]:
# model_name = 'model_14'

In [17]:
# torch.save(model, './Model/'+model_name )

In [18]:
# expert_model = torch.load('./Model/'+model_name)

In [19]:
# expert_test_rewards = []
# for i in range(5): 
# #     env = gym.wrappers.Monitor(env, 'test_video'+str(i), video_callable=lambda episode_id: True)
#     expert_test_reward = test_env(expert_model, [0, 0, 0], False)
#     expert_test_rewards.append(expert_test_reward)
#     print ('test {0}, total_reward from '+model_name+' load model: {1}'.format(i+1, expert_test_reward))

# # print ('mean expert test reward: ', np.mean(expert_test_rewards))